In [2]:
import pandas as pd

In [5]:
nutrition_and_obesity_data =  pd.read_csv('../data/provided-data-raw/Nutrition_Physical_Activity_and_Obesity_Data.csv')

# remove obviously useless or redundant data
nutrition_and_obesity_data = nutrition_and_obesity_data.dropna(subset=['Data_Value'])
nutrition_and_obesity_data = nutrition_and_obesity_data.drop(['YearEnd', 'Data_Value_Unit', 'Data_Value_Type', 'Data_Value_Alt',
                                  'Data_Value_Footnote_Symbol', 'Data_Value_Footnote'], axis=1)
nutrition_and_obesity_data = nutrition_and_obesity_data.rename(columns={'YearStart': 'Year'})

# need to figure out how we want to further wrangle the data
nutrition_and_obesity_data.to_csv('../data/provided-data-clean/nutrition-and-obesity-cleaned.csv')

/var/folders/73/mqvvsvdn6k10b__2ljzg54s00000gn/T/ipykernel_50305/1406095656.py:1: DtypeWarning: Columns (18,19,21,22) have mixed types. Specify dtype option on import or set low_memory=False.
  nutrition_and_obesity_data =  pd.read_csv('../data/provided-data-raw/Nutrition_Physical_Activity_and_Obesity_Data.csv')


In [13]:
# extract specifically obesity data for students
nutrition_and_obesity_data =  pd.read_csv('../data/data-clean/nutrition-and-obesity-cleaned.csv')
obesity_data = nutrition_and_obesity_data[nutrition_and_obesity_data['Question'] == 
                                          'Percent of adults aged 18 years and older who have obesity']

# obesity_state_2011 = obesity_data[obesity_data['Year'] == 2011].groupby('LocationAbbr').agg({'Data_Value': 'mean'}).reset_index()
# obesity_state_2011 = obesity_state_2011.rename(columns={'Data_Value': 'Mean_Obesity_Pct'})

# obesity_state_2015 = obesity_data[obesity_data['Year'] == 2015].groupby('LocationAbbr').agg({'Data_Value': 'mean'}).reset_index()
# obesity_state_2015 = obesity_state_2015.rename(columns={'Data_Value': 'Mean_Obesity_Pct'})

# obesity_state_2011.to_csv('../data/provided-data-clean/obesity-state-2011-cleaned.csv')
# obesity_state_2015.to_csv('../data/provided-data-clean/obesity-state-2015-cleaned.csv')

obesity_state_2019 = obesity_data[obesity_data['Year'] == 2019].groupby('LocationAbbr').agg({'Data_Value': 'mean'}).reset_index()
obesity_state_2019 = obesity_state_2019.rename(columns={'Data_Value': 'Mean_Obesity_Pct'})

obesity_state_2019.to_csv('../data/data-state-for-processing/obesity-state-2019-cleaned.csv')


/var/folders/xr/0vzl322108z0xw4wh8532jj80000gn/T/ipykernel_90947/493209323.py:2: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  nutrition_and_obesity_data =  pd.read_csv('../data/data-clean/nutrition-and-obesity-cleaned.csv')


In [6]:
# clean food evironment atlas

food_environ_data = pd.read_csv('../data/other-data-raw/food-environment-atlas-raw/state_county_data_raw.csv')
variable_names = pd.read_csv('../data/other-data-raw/food-environment-atlas-raw/variable_list_pruned.csv')

valid_variable_codes = variable_names['Variable Code'].tolist()
print(valid_variable_codes)

# filter the rows based on the variable codes
filtered_data = food_environ_data[food_environ_data['Variable_Code'].isin(valid_variable_codes)]
# drop the FIPS column
filtered_data = filtered_data.drop(columns=['FIPS'])

filtered_data.to_csv('../data/other-data-clean/food_environ_county_data_clean.csv', index=False)

['LACCESS_POP10', 'LACCESS_POP15', 'LACCESS_LOWI10', 'LACCESS_LOWI15', 'LACCESS_HHNV10', 'LACCESS_HHNV15', 'LACCESS_SNAP15', 'LACCESS_CHILD10', 'LACCESS_CHILD15', 'LACCESS_SENIORS10', 'LACCESS_SENIORS15', 'LACCESS_WHITE15', 'LACCESS_BLACK15', 'LACCESS_HISP15', 'LACCESS_NHASIAN15', 'LACCESS_NHNA15', 'LACCESS_NHPI15', 'LACCESS_MULTIR15', 'GROC11', 'GROC16', 'GROCPTH11', 'GROCPTH16', 'SUPERC11', 'SUPERC16', 'SUPERCPTH11', 'SUPERCPTH16', 'CONVS11', 'CONVS16', 'CONVSPTH11', 'CONVSPTH16', 'SPECS11', 'SPECS16', 'SPECSPTH11', 'SPECSPTH16', 'SNAPS12', 'SNAPS17', 'SNAPSPTH12', 'SNAPSPTH17', 'WICS11', 'WICS16', 'WICSPTH11', 'WICSPTH16', 'FFR11', 'FFR16', 'FFRPTH11', 'FFRPTH16', 'FSR11', 'FSR16', 'FSRPTH11', 'FSRPTH16', 'PC_FFRSALES07', 'PC_FFRSALES12', 'PC_FSRSALES07', 'PC_FSRSALES12', 'FOOD_BANKS18', 'FMRKT13', 'FMRKT18', 'FMRKTPTH13', 'FMRKTPTH18', 'FMRKT_FRVEG18', 'FMRKT_ANMLPROD18', 'FMRKT_BAKED18', 'PCT_DIABETES_ADULTS08', 'PCT_DIABETES_ADULTS13', 'PCT_OBESE_ADULTS12', 'PCT_OBESE_ADULTS17', 

In [10]:
# extract state data from county data
cleaned_county_data = pd.read_csv('../data/other-data-clean/food_environ_county_data_clean.csv')
variable_names = pd.read_csv('../data/other-data-raw/food-environment-atlas-raw/variable_list_primary.csv')

# only include count variables
valid_variable_codes = variable_names[variable_names['Units'].str.contains('Count', na=False)]['Variable Code'].tolist()
filtered_data = cleaned_county_data[cleaned_county_data['Variable_Code'].isin(valid_variable_codes)]

# group by 'State' and 'Variable_Code' columns and sum the 'Value' column
state_data = filtered_data.groupby(['State', 'Variable_Code'], as_index=False)['Value'].sum()

# add variable name column to improve readability
variable_code_to_name = variable_names.set_index('Variable Code')['Variable Name'].to_dict()
state_data['Variable Name'] = state_data['Variable_Code'].map(variable_code_to_name)
state_data = state_data[['State', 'Variable Name', 'Variable_Code', 'Value']]

state_data.to_csv('../data/other-data-clean/food_environ_state_data_clean.csv', index=False)

In [6]:
# unique questions in the cleaned nutrition and obesity

data = pd.read_csv('../data/provided-data-clean/nutrition-and-obesity-cleaned.csv')

unique_questions = data['Question'].unique()
print(unique_questions)

['Percent of students in grades 9-12 who achieve 1 hour or more of moderate-and/or vigorous-intensity physical activity daily'
 'Percent of students in grades 9-12 who have obesity'
 'Percent of students in grades 9-12 who participate in daily physical education'
 'Percent of students in grades 9-12 who have an overweight classification'
 'Percent of students in grades 9-12 who drank regular soda/pop at least one time per day'
 'Percent of students in grades 9-12 who consume vegetables less than 1 time daily'
 'Percent of students in grades 9-12 who consume fruit less than 1 time daily'
 'Percent of students in grades 9-12 watching 3 or more hours of television each school day'
 'Percent of adults who engage in no leisure-time physical activity'
 'Percent of adults aged 18 years and older who have obesity'
 'Percent of adults aged 18 years and older who have an overweight classification'
 'Percent of adults who achieve at least 300 minutes a week of moderate-intensity aerobic physical 

/var/folders/xr/0vzl322108z0xw4wh8532jj80000gn/T/ipykernel_84111/3325909735.py:3: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('../data/provided-data-clean/nutrition-and-obesity-cleaned.csv')


In [13]:
variable_names = pd.read_csv('../data/other-data-raw/food-environment-atlas-raw/variable_list_primary.csv')

print(variable_names['Variable Name'].tolist())

['Population, low access to store, 2010', 'Population, low access to store, 2015', 'Low income & low access to store, 2010', 'Low income & low access to store, 2015', 'Households, no car & low access to store, 2010', 'Households, no car & low access to store, 2015', 'Grocery stores, 2011', 'Grocery stores, 2016', 'Convenience stores, 2011', 'Convenience stores, 2016', 'Fast-food restaurants, 2011', 'Fast-food restaurants, 2016', "Farmers' markets, 2013", "Farmers' markets, 2018"]


In [9]:
# extract median household income

spreadsheet = pd.read_csv('../data/data-raw/acs_5yr_est_selected_economic_characteristics_2010-2022.csv')
print(spreadsheet)

spreadsheet['Label (Grouping)'] = spreadsheet['Label (Grouping)'].str.strip()

# Filter the rows where "Label (Grouping)" is "Median household income (dollars)" and "Year" is 2019
filtered_data = spreadsheet[(spreadsheet['Label (Grouping)'] == 'Median household income (dollars)') & (spreadsheet['Year'] == 2019)]
print(filtered_data)
filtered_data = filtered_data[['State', 'Estimate', 'Year']]

# Save the filtered data to a new CSV file
filtered_data.to_csv('../data/data-state-for-processing/median_household_income_2019.csv', index=False)

                              Label (Grouping)  \
0                            EMPLOYMENT STATUS   
1                 Population 16 years and over   
2                               In labor force   
3                         Civilian labor force   
4                                     Employed   
...                                        ...   
98015                        18 years and over   
98016                           18 to 64 years   
98017                        65 years and over   
98018                       People in families   
98019  Unrelated individuals 15 years and over   

                                                Category        State  \
0                                                 Header      Alabama   
1                                      EMPLOYMENT STATUS      Alabama   
2                                      EMPLOYMENT STATUS      Alabama   
3                                      EMPLOYMENT STATUS      Alabama   
4                                 